In [1]:
import numpy as np
import pandas as pd
import pandas as pd
from collections import Counter, defaultdict
import re
import ast

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
picks = pd.read_parquet('../../Data/2023_2025_pick_trades.parquet')
# Normalize franchise ownership
OWNER_REMAP = {
    "Weeks": "Gio"
}

picks = picks.copy()

picks["receiving_owner"] = picks["receiving_owner"].replace(OWNER_REMAP)
picks["giving_owner"] = picks["giving_owner"].replace(OWNER_REMAP)


In [3]:
WEEKS = (
    ['pre-season']
    + [str(i) for i in range(1, 15)]
    + ['offseason']
)

week_order = {w: i for i, w in enumerate(WEEKS)}
ROUNDS = [1, 2, 3, 4]

In [4]:
def safe_parse_pick_list(x):
    if isinstance(x, (list, tuple, set, np.ndarray)):
        return [str(i) for i in x if i]

    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []

    if isinstance(x, str):
        x = x.strip()
        if x in ("", "[]"):
            return []
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, (list, tuple, set)):
                return [str(i) for i in parsed if i]
        except Exception:
            return [i.strip().strip("'\"") for i in x.split(",") if i.strip()]

    return [str(x)]


In [5]:
trade_effects = []

for _, row in picks.iterrows():
    season = int(row['season'])
    week = str(row['week'])
    ro = row['receiving_owner']
    go = row['giving_owner']

    ro_recv = Counter(safe_parse_pick_list(row.get('ro_picks_received', [])))
    ro_gave = Counter(safe_parse_pick_list(row.get('ro_picks_gave', [])))
    go_recv = Counter(safe_parse_pick_list(row.get('go_picks_received', [])))
    go_gave = Counter(safe_parse_pick_list(row.get('go_picks_gave', [])))

    trade_effects.append({
        'owner': ro,
        'season': season,
        'week': week,
        'delta': ro_recv - ro_gave
    })
    trade_effects.append({
        'owner': go,
        'season': season,
        'week': week,
        'delta': go_recv - go_gave
    })

trade_effects_df = pd.DataFrame(trade_effects)


In [6]:
trade_effects_df['week_sort'] = trade_effects_df['week'].map(week_order)
trade_effects_df = trade_effects_df.sort_values(
    ['season', 'week_sort']
).drop(columns='week_sort')


In [7]:
owners = sorted(set(picks['receiving_owner']).union(picks['giving_owner']))

ledger = {o: Counter() for o in owners}

# 2023 inception: everyone gets 2024–2026
for o in owners:
    for y in [2024, 2025, 2026]:
        for r in ROUNDS:
            ledger[o][f"{y}_{r}"] += 1


In [8]:
records = []

def add_new_season_picks(season):
    new_year = season + 3
    for o in owners:
        for r in ROUNDS:
            ledger[o][f"{new_year}_{r}"] += 1

current_season = 2023

for (season, week), chunk in trade_effects_df.groupby(['season', 'week'], sort=False):

    # add new picks at start of season
    if season != current_season:
        add_new_season_picks(season)
        current_season = season

    # apply all trades in this bucket
    for _, row in chunk.iterrows():
        owner = row['owner']
        ledger[owner] += row['delta']

    # snapshot
    for o in owners:
        snap = {
            'owner': o,
            'season': season,
            'week': week
        }
        for y in range(2024, 2029):
            for r in ROUNDS:
                snap[f"{y}_{r}"] = ledger[o].get(f"{y}_{r}", 0)
        records.append(snap)


In [9]:
pick_time_series_clean = pd.DataFrame(records)

pick_time_series_clean['week_sort'] = pick_time_series_clean['week'].map(week_order)

pick_time_series_clean = (
    pick_time_series_clean
    .sort_values(['owner', 'season', 'week_sort'])
    .drop(columns='week_sort')
    .reset_index(drop=True)
)


In [10]:
pick_time_series_clean[pick_time_series_clean['owner'] == 'Jose']

,owner,season,week,2024_1,2024_2,2024_3,2024_4,2025_1,2025_2,2025_3,2025_4,2026_1,2026_2,2026_3,2026_4,2027_1,2027_2,2027_3,2027_4,2028_1,2028_2,2028_3,2028_4
180,Jose,2023,pre-season,1,1,1,2,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
181,Jose,2023,1,1,1,1,2,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
182,Jose,2023,2,1,2,1,3,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
183,Jose,2023,3,1,2,1,3,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
184,Jose,2023,4,1,3,1,3,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
185,Jose,2023,5,1,3,1,3,2,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0
186,Jose,2023,6,1,4,1,3,2,3,2,1,1,1,1,1,0,0,0,0,0,0,0,0
187,Jose,2023,7,1,4,1,3,2,3,2,1,1,1,1,1,0,0,0,0,0,0,0,0
188,Jose,2023,8,1,4,1,3,2,3,2,1,1,1,1,1,0,0,0,0,0,0,0,0
189,Jose,2023,9,1,5,1,3,2,3,3,1,1,1,1,1,0,0,0,0,0,0,0,0


In [11]:
pick_time_series_clean_zeroed = pick_time_series_clean.copy()

pick_cols = [c for c in pick_time_series_clean.columns if re.match(r'\d+_\d+', c)]

for idx, row in pick_time_series_clean_zeroed.iterrows():
    season = int(row['season'])
    for col in pick_cols:
        pick_year = int(col.split('_')[0])
        if pick_year <= season:
            pick_time_series_clean_zeroed.at[idx, col] = 0


In [14]:
owners = pick_time_series_clean_zeroed['owner'].unique()
seasons = pick_time_series_clean_zeroed['season'].unique()
weeks = WEEKS  

import itertools

full_index = pd.DataFrame(
    list(itertools.product(owners, seasons, weeks)),
    columns=['owner', 'season', 'week']
)


In [15]:
pick_time_series_full = full_index.merge(
    pick_time_series_clean_zeroed,
    on=['owner', 'season', 'week'],
    how='left'
)


In [16]:
pick_time_series_full['week_sort'] = pick_time_series_full['week'].map(week_order)
pick_time_series_full = pick_time_series_full.sort_values(
    by=['owner', 'season', 'week_sort']
).drop(columns='week_sort').reset_index(drop=True)


In [17]:
pick_cols = [c for c in pick_time_series_full.columns if re.match(r'\d+_\d+', c)]

pick_time_series_full[pick_cols] = pick_time_series_full.groupby(
    ['owner', 'season']
)[pick_cols].ffill().fillna(0).astype(int)


In [19]:
pick_time_series_full[pick_time_series_full['owner'] == 'Jose']

,owner,season,week,2024_1,2024_2,2024_3,2024_4,2025_1,2025_2,2025_3,2025_4,2026_1,2026_2,2026_3,2026_4,2027_1,2027_2,2027_3,2027_4,2028_1,2028_2,2028_3,2028_4
288,Jose,2023,pre-season,1,1,1,2,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
289,Jose,2023,1,1,1,1,2,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
290,Jose,2023,2,1,2,1,3,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
291,Jose,2023,3,1,2,1,3,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
292,Jose,2023,4,1,3,1,3,2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
293,Jose,2023,5,1,3,1,3,2,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0
294,Jose,2023,6,1,4,1,3,2,3,2,1,1,1,1,1,0,0,0,0,0,0,0,0
295,Jose,2023,7,1,4,1,3,2,3,2,1,1,1,1,1,0,0,0,0,0,0,0,0
296,Jose,2023,8,1,4,1,3,2,3,2,1,1,1,1,1,0,0,0,0,0,0,0,0
297,Jose,2023,9,1,5,1,3,2,3,3,1,1,1,1,1,0,0,0,0,0,0,0,0


In [20]:
pick_time_series_full.to_parquet('../../Data/Historical_Picks_by_Week.parquet')